In [238]:
import numpy as np
import pandas as pd
import random as rd

In [239]:
# Traigo la base de datos
df = pd.read_csv('../Datos/tarifas.csv',sep =';')                       #leo los datos de las tarifas sacadas a mano
consumos_modelo = pd.read_csv('../Datos/consumos_modelo.csv')

In [240]:
df.head()

,id,compania,tarifa,tp_punta_euros/kwh/dia,tp_valle_euros/kwh/dia,peaje_tran_dist_pot_punta,peaje_tran_dist_pot_valle,peaje_ener_valle_tdc,peaje_ener_llano_tdc,peaje_ener_punta_tdc,termino_consumo_euros/kwh/hora,tc_valle_euros/kwh/hora,tc_llano_euros/kwh/hora,tc_punta_euros/kwh/hora,tc_8h_euros/kwh/hora,tc_16h_euros/kwh/hora,mercado,tipo,otros,alquiler_equipo_euros/dia,impuesto_actual,bono_social_euro/dia,margen_euros/kWh/dia
0,0,Repsol,Mis 10 Horas,0.068192,0.068192,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.0,0.0,0.0,0.086415,0.17283,Libre,DH,0.0,0.026557,5.11,0.0,NaN
1,1,Iberdrola,Plan Elige 8 Horas,0.108192,0.042411,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.0,0.0,0.0,0.096700,0.24100,Libre,DH,0.0,0.026557,5.11,0.0,NaN
2,2,Iberdrola,Plan Noche,0.108192,0.042411,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.0,0.0,0.0,0.109000,0.23900,Libre,DH,0.0,0.026557,5.11,0.0,NaN
3,3,Iberdrola,Plan Familias,0.108192,0.042411,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.0,0.0,0.0,0.103000,0.18900,Libre,DH,0.0,0.026557,5.11,0.0,NaN
4,4,Iberdrola,Plan Verano,0.108192,0.042411,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.0,0.0,0.0,0.099000,0.21900,Libre,DH,0.0,0.026557,5.11,0.0,NaN


In [241]:
# Uno los datos sinteticos con las tarifas
clientes_tarifas = pd.merge(consumos_modelo, df, left_on='id_tarifa', right_on='id', how='left')            

In [242]:
# Para que se vean todas las columnas
pd.set_option('display.max_columns', None)

In [243]:
clientes_tarifas                        #db con facturas y consumos ficticias

,Unnamed: 0,id_tarifa,consumo_kWh,%consumo_valle,%consumo_llano,%consumo_punta,%consumo_8h,%consumo_16h,id,compania,tarifa,tp_punta_euros/kwh/dia,tp_valle_euros/kwh/dia,peaje_tran_dist_pot_punta,peaje_tran_dist_pot_valle,peaje_ener_valle_tdc,peaje_ener_llano_tdc,peaje_ener_punta_tdc,termino_consumo_euros/kwh/hora,tc_valle_euros/kwh/hora,tc_llano_euros/kwh/hora,tc_punta_euros/kwh/hora,tc_8h_euros/kwh/hora,tc_16h_euros/kwh/hora,mercado,tipo,otros,alquiler_equipo_euros/dia,impuesto_actual,bono_social_euro/dia,margen_euros/kWh/dia
0,0,0,206,29,70,1,22,78,0,Repsol,Mis 10 Horas,0.068192,0.068192,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.000000,0.000000,0.000000,0.086415,0.17283,Libre,DH,0.00,0.026557,5.11,0.0,NaN
1,1,0,276,11,51,38,67,33,0,Repsol,Mis 10 Horas,0.068192,0.068192,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.000000,0.000000,0.000000,0.086415,0.17283,Libre,DH,0.00,0.026557,5.11,0.0,NaN
2,2,0,220,23,47,30,35,65,0,Repsol,Mis 10 Horas,0.068192,0.068192,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.000000,0.000000,0.000000,0.086415,0.17283,Libre,DH,0.00,0.026557,5.11,0.0,NaN
3,3,0,198,21,76,3,55,45,0,Repsol,Mis 10 Horas,0.068192,0.068192,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.000000,0.000000,0.000000,0.086415,0.17283,Libre,DH,0.00,0.026557,5.11,0.0,NaN
4,4,0,215,17,24,59,77,23,0,Repsol,Mis 10 Horas,0.068192,0.068192,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.000000,0.000000,0.000000,0.086415,0.17283,Libre,DH,0.00,0.026557,5.11,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2179,2179,103,212,87,7,6,70,30,103,Aduriz Energia,coste + fija,0.080039,0.003141,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.093282,0.129895,0.179284,0.000000,0.00000,Libre,TRAMOS,3.72,0.026557,5.11,0.0,NaN
2180,2180,103,146,16,31,53,1,99,103,Aduriz Energia,coste + fija,0.080039,0.003141,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.093282,0.129895,0.179284,0.000000,0.00000,Libre,TRAMOS,3.72,0.026557,5.11,0.0,NaN
2181,2181,103,123,1,50,49,84,16,103,Aduriz Energia,coste + fija,0.080039,0.003141,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.093282,0.129895,0.179284,0.000000,0.00000,Libre,TRAMOS,3.72,0.026557,5.11,0.0,NaN
2182,2182,103,246,0,13,87,81,19,103,Aduriz Energia,coste + fija,0.080039,0.003141,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.093282,0.129895,0.179284,0.000000,0.00000,Libre,TRAMOS,3.72,0.026557,5.11,0.0,NaN


In [244]:
# comprobamos por tarifas si el merge es como debe

clientes_tarifas[clientes_tarifas['id_tarifa'] == 12]

,Unnamed: 0,id_tarifa,consumo_kWh,%consumo_valle,%consumo_llano,%consumo_punta,%consumo_8h,%consumo_16h,id,compania,tarifa,tp_punta_euros/kwh/dia,tp_valle_euros/kwh/dia,peaje_tran_dist_pot_punta,peaje_tran_dist_pot_valle,peaje_ener_valle_tdc,peaje_ener_llano_tdc,peaje_ener_punta_tdc,termino_consumo_euros/kwh/hora,tc_valle_euros/kwh/hora,tc_llano_euros/kwh/hora,tc_punta_euros/kwh/hora,tc_8h_euros/kwh/hora,tc_16h_euros/kwh/hora,mercado,tipo,otros,alquiler_equipo_euros/dia,impuesto_actual,bono_social_euro/dia,margen_euros/kWh/dia
252,252,12,227,54,4,42,50,50,12,Endesa,ConectaEndesa,0.099703,0.032791,0.008192,0.000527,0.002752,0.027963,0.076974,0.0957,0.0,0.0,0.0,0.0,0.0,Libre,FIJO,0.0,0.026557,5.11,0.0,NaN
253,253,12,183,4,65,31,40,60,12,Endesa,ConectaEndesa,0.099703,0.032791,0.008192,0.000527,0.002752,0.027963,0.076974,0.0957,0.0,0.0,0.0,0.0,0.0,Libre,FIJO,0.0,0.026557,5.11,0.0,NaN
254,254,12,217,11,26,63,44,56,12,Endesa,ConectaEndesa,0.099703,0.032791,0.008192,0.000527,0.002752,0.027963,0.076974,0.0957,0.0,0.0,0.0,0.0,0.0,Libre,FIJO,0.0,0.026557,5.11,0.0,NaN
255,255,12,150,52,5,43,72,28,12,Endesa,ConectaEndesa,0.099703,0.032791,0.008192,0.000527,0.002752,0.027963,0.076974,0.0957,0.0,0.0,0.0,0.0,0.0,Libre,FIJO,0.0,0.026557,5.11,0.0,NaN
256,256,12,223,4,65,31,100,0,12,Endesa,ConectaEndesa,0.099703,0.032791,0.008192,0.000527,0.002752,0.027963,0.076974,0.0957,0.0,0.0,0.0,0.0,0.0,Libre,FIJO,0.0,0.026557,5.11,0.0,NaN
257,257,12,212,18,16,66,64,36,12,Endesa,ConectaEndesa,0.099703,0.032791,0.008192,0.000527,0.002752,0.027963,0.076974,0.0957,0.0,0.0,0.0,0.0,0.0,Libre,FIJO,0.0,0.026557,5.11,0.0,NaN
258,258,12,106,60,9,31,22,78,12,Endesa,ConectaEndesa,0.099703,0.032791,0.008192,0.000527,0.002752,0.027963,0.076974,0.0957,0.0,0.0,0.0,0.0,0.0,Libre,FIJO,0.0,0.026557,5.11,0.0,NaN
259,259,12,168,55,11,34,11,89,12,Endesa,ConectaEndesa,0.099703,0.032791,0.008192,0.000527,0.002752,0.027963,0.076974,0.0957,0.0,0.0,0.0,0.0,0.0,Libre,FIJO,0.0,0.026557,5.11,0.0,NaN
260,260,12,246,53,46,1,15,85,12,Endesa,ConectaEndesa,0.099703,0.032791,0.008192,0.000527,0.002752,0.027963,0.076974,0.0957,0.0,0.0,0.0,0.0,0.0,Libre,FIJO,0.0,0.026557,5.11,0.0,NaN
261,261,12,138,5,64,31,44,56,12,Endesa,ConectaEndesa,0.099703,0.032791,0.008192,0.000527,0.002752,0.027963,0.076974,0.0957,0.0,0.0,0.0,0.0,0.0,Libre,FIJO,0.0,0.026557,5.11,0.0,NaN


In [245]:
clientes_tarifas.drop(columns = ['id','Unnamed: 0'],inplace=True)

In [246]:
clientes_tarifas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2184 entries, 0 to 2183
Data columns (total 29 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id_tarifa                       2184 non-null   int64  
 1   consumo_kWh                     2184 non-null   int64  
 2   %consumo_valle                  2184 non-null   int64  
 3   %consumo_llano                  2184 non-null   int64  
 4   %consumo_punta                  2184 non-null   int64  
 5   %consumo_8h                     2184 non-null   int64  
 6   %consumo_16h                    2184 non-null   int64  
 7   compania                        2184 non-null   object 
 8   tarifa                          2184 non-null   object 
 9   tp_punta_euros/kwh/dia          2184 non-null   float64
 10  tp_valle_euros/kwh/dia          2184 non-null   float64
 11  peaje_tran_dist_pot_punta       2184 non-null   float64
 12  peaje_tran_dist_pot_valle       21

In [247]:
int = clientes_tarifas.columns[2:7]

In [248]:
for x in int:
    if clientes_tarifas[x].dtype == 'object':
        clientes_tarifas[x] = clientes_tarifas[x].astype('int32') 


In [249]:
clientes_tarifas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2184 entries, 0 to 2183
Data columns (total 29 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id_tarifa                       2184 non-null   int64  
 1   consumo_kWh                     2184 non-null   int64  
 2   %consumo_valle                  2184 non-null   int64  
 3   %consumo_llano                  2184 non-null   int64  
 4   %consumo_punta                  2184 non-null   int64  
 5   %consumo_8h                     2184 non-null   int64  
 6   %consumo_16h                    2184 non-null   int64  
 7   compania                        2184 non-null   object 
 8   tarifa                          2184 non-null   object 
 9   tp_punta_euros/kwh/dia          2184 non-null   float64
 10  tp_valle_euros/kwh/dia          2184 non-null   float64
 11  peaje_tran_dist_pot_punta       2184 non-null   float64
 12  peaje_tran_dist_pot_valle       21

In [250]:
clientes_tarifas

,id_tarifa,consumo_kWh,%consumo_valle,%consumo_llano,%consumo_punta,%consumo_8h,%consumo_16h,compania,tarifa,tp_punta_euros/kwh/dia,tp_valle_euros/kwh/dia,peaje_tran_dist_pot_punta,peaje_tran_dist_pot_valle,peaje_ener_valle_tdc,peaje_ener_llano_tdc,peaje_ener_punta_tdc,termino_consumo_euros/kwh/hora,tc_valle_euros/kwh/hora,tc_llano_euros/kwh/hora,tc_punta_euros/kwh/hora,tc_8h_euros/kwh/hora,tc_16h_euros/kwh/hora,mercado,tipo,otros,alquiler_equipo_euros/dia,impuesto_actual,bono_social_euro/dia,margen_euros/kWh/dia
0,0,206,29,70,1,22,78,Repsol,Mis 10 Horas,0.068192,0.068192,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.000000,0.000000,0.000000,0.086415,0.17283,Libre,DH,0.00,0.026557,5.11,0.0,NaN
1,0,276,11,51,38,67,33,Repsol,Mis 10 Horas,0.068192,0.068192,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.000000,0.000000,0.000000,0.086415,0.17283,Libre,DH,0.00,0.026557,5.11,0.0,NaN
2,0,220,23,47,30,35,65,Repsol,Mis 10 Horas,0.068192,0.068192,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.000000,0.000000,0.000000,0.086415,0.17283,Libre,DH,0.00,0.026557,5.11,0.0,NaN
3,0,198,21,76,3,55,45,Repsol,Mis 10 Horas,0.068192,0.068192,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.000000,0.000000,0.000000,0.086415,0.17283,Libre,DH,0.00,0.026557,5.11,0.0,NaN
4,0,215,17,24,59,77,23,Repsol,Mis 10 Horas,0.068192,0.068192,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.000000,0.000000,0.000000,0.086415,0.17283,Libre,DH,0.00,0.026557,5.11,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2179,103,212,87,7,6,70,30,Aduriz Energia,coste + fija,0.080039,0.003141,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.093282,0.129895,0.179284,0.000000,0.00000,Libre,TRAMOS,3.72,0.026557,5.11,0.0,NaN
2180,103,146,16,31,53,1,99,Aduriz Energia,coste + fija,0.080039,0.003141,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.093282,0.129895,0.179284,0.000000,0.00000,Libre,TRAMOS,3.72,0.026557,5.11,0.0,NaN
2181,103,123,1,50,49,84,16,Aduriz Energia,coste + fija,0.080039,0.003141,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.093282,0.129895,0.179284,0.000000,0.00000,Libre,TRAMOS,3.72,0.026557,5.11,0.0,NaN
2182,103,246,0,13,87,81,19,Aduriz Energia,coste + fija,0.080039,0.003141,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.093282,0.129895,0.179284,0.000000,0.00000,Libre,TRAMOS,3.72,0.026557,5.11,0.0,NaN


In [251]:
tarifas = pd.read_csv('../Datos/tarifas.csv',sep = ';')         #leo los datos sacados a mano

In [252]:
calculo = tarifas.copy()                                        #hago copia

In [253]:
calculo['margen_euros/kWh/dia'] = calculo['margen_euros/kWh/dia'].fillna(0)         #rellena con 0 la columna margen_euros

In [254]:
datos_formulario = pd.read_csv(r'../Datos/datos_tarifas.csv')                       #leo los datos del formulario
datos_formulario

,Discriminacion horaria,Termino de potencia en valle,Termino de potencia en punta,Consumo en punta (kWh),Consumo en llano (kWh),Consumo en valle (kWh),Consumo total (kWh),Bono social,Total factura,Discriminación horaria
0,NaN,3.3,3.3,30.0,15.0,0.0,45.0,NO,150.00,SI
1,NaN,3.3,3.3,40.0,20.0,20.0,80.0,NO,210.00,SI
2,NaN,3.3,3.3,40.0,20.0,20.0,80.0,SI,210.00,SI
3,NaN,3.3,3.3,40.0,20.0,20.0,80.0,NO,210.00,NO
4,NaN,3.3,3.3,300.0,300.0,300.0,500.0,SI,0.00,SI
5,NaN,3.3,3.3,100.0,100.0,100.0,300.0,SI,0.00,SI
6,NaN,3.3,3.3,200.0,200.0,200.0,300.0,SI,0.00,NO
7,NaN,3.3,3.3,0.0,0.0,0.0,10.0,SI,150.00,NO
8,NaN,3.3,3.3,0.0,0.0,0.0,10000.0,SI,1500.00,NO
9,NaN,3.3,3.3,300.0,300.0,300.0,10000.0,SI,1500.00,NO


In [255]:
formulario = datos_formulario.tail(1).to_dict(orient = 'records')[0]                #convierte el ultimo registro

formulario['periodo'] = 30

In [256]:
formulario

{'Discriminacion horaria': nan,
 'Termino de potencia en valle': 3.3,
 'Termino de potencia en punta': 3.3,
 'Consumo en punta (kWh)': 100.0,
 'Consumo en llano (kWh)': 100.0,
 'Consumo en valle (kWh)': 100.0,
 'Consumo total (kWh)': 300.0,
 'Bono social': 'SI',
 'Total factura': 10.0,
 'Discriminación horaria': 'SI',
 'periodo': 30}

In [273]:
calculo['consumo_valle'] = formulario['Consumo en valle (kWh)']
calculo['consumo_llano'] = formulario['Consumo en llano (kWh)']
calculo['consumo_punta'] = formulario['Consumo en punta (kWh)']
calculo['total_actual'] = formulario['Total factura']

In [258]:
# calculoS DE TARIFA, DEFINIMOS FUNCIONES

#PVPC

def pvpc(fila,formulario):

  calculo = fila.to_dict()  
  #POTENCIA

  #transporte y distribucion
  punta_tp = formulario['Termino de potencia en punta']*formulario['periodo']*calculo['peaje_tran_dist_pot_punta']
  valle_tp = formulario['Termino de potencia en valle']*formulario['periodo']*calculo['peaje_tran_dist_pot_valle']

  #cargos
  punta_cp = formulario['Termino de potencia en punta']*formulario['periodo']*calculo['tp_punta_euros/kwh/dia']
  valle_cp = formulario['Termino de potencia en valle']*formulario['periodo']*calculo['tp_valle_euros/kwh/dia']

  #margen. Se cobra solo sobre potencia punta en el caso de tener dos potencias contratadas
  margen = formulario['Termino de potencia en punta']*formulario['periodo']*calculo['margen_euros/kWh/dia']

  potencia = punta_tp + punta_cp + valle_tp + valle_cp + margen

  # ENERGIA

  #transporte, distribucion y cargos
  valle_te = formulario['Consumo en valle (kWh)']*calculo['peaje_ener_valle_tdc']
  llano_te = formulario['Consumo en llano (kWh)']*calculo['peaje_ener_llano_tdc']
  punta_te = formulario['Consumo en punta (kWh)']*calculo['peaje_ener_punta_tdc']

  #coste de la energia 
  valle_ce = formulario['Consumo en valle (kWh)']*calculo['tc_valle_euros/kwh/hora']
  llano_ce = formulario['Consumo en llano (kWh)']*calculo['tc_llano_euros/kwh/hora']
  punta_ce = formulario['Consumo en punta (kWh)']*calculo['tc_punta_euros/kwh/hora']

  consumo = valle_ce + llano_ce + punta_ce #solo energia consumida
 
  energia = valle_te + llano_te + punta_te + potencia + consumo #energia consumida mas peajes

  #BONO SOCIAL

  bono = formulario['periodo']*calculo['bono_social_euro/dia']

  if formulario['Bono social'] == 'Si':

    base = (potencia + consumo)*0.6 
    subtotal = (potencia + energia) - base

  else:
    subtotal = potencia + energia + bono
    
  imp_elec = subtotal * (calculo['impuesto_actual']/100)

  contador = formulario['periodo'] * calculo['alquiler_equipo_euros/dia']

  total = subtotal + imp_elec + contador

  total_factura = total * 1.10

  return total_factura

#falta el bono y el descuento de familia numerosa

#FIJA

def fija(fila,formulario):

  calculo = fila.to_dict() 

  pot_v = formulario['Termino de potencia en valle'] *formulario['periodo']*calculo['tp_valle_euros/kwh/dia']
  pot_p = formulario['Termino de potencia en punta'] *formulario['periodo']*calculo['tp_punta_euros/kwh/dia']

  energia = formulario['Consumo total (kWh)']*calculo['termino_consumo_euros/kwh/hora']

  subtotal = pot_v + pot_p + energia

  impuesto = subtotal * (calculo['impuesto_actual']/100)

  contador = formulario['periodo']*calculo['alquiler_equipo_euros/dia']

  total = subtotal + impuesto + contador

  total_factura = total * 1.10

  return total_factura


#TRAMOS

def tramos(fila,formulario):

  calculo = fila.to_dict() 

  pot_v = formulario['Termino de potencia en valle'] *formulario['periodo']*calculo['tp_valle_euros/kwh/dia']
  pot_p = formulario['Termino de potencia en punta'] *formulario['periodo']*calculo['tp_punta_euros/kwh/dia']

  potencia = pot_v + pot_p

  valle_c = formulario['Consumo en valle (kWh)']*calculo['tc_valle_euros/kwh/hora']
  llano_c = formulario['Consumo en llano (kWh)']*calculo['tc_llano_euros/kwh/hora']
  punta_c = formulario['Consumo en punta (kWh)']*calculo['tc_punta_euros/kwh/hora']

  energia = valle_c + llano_c + punta_c

  subtotal = potencia + energia

  impuesto = subtotal * (calculo['impuesto_actual']/100)

  contador = formulario['periodo']*calculo['alquiler_equipo_euros/dia']

  total = subtotal + impuesto + contador

  total_factura = total * 1.10

  return total_factura

#DISCRIMINACION HORARIA

def dh(fila,formulario):

  calculo = fila.to_dict() 

  periodo_bon = formulario['Consumo en valle (kWh)'] 
  periodo_nbon = formulario['Consumo en llano (kWh)'] + formulario['Consumo en punta (kWh)']

  pot_v = formulario['Termino de potencia en valle'] *formulario['periodo']*calculo['tp_valle_euros/kwh/dia']
  pot_p = formulario['Termino de potencia en punta'] *formulario['periodo']*calculo['tp_punta_euros/kwh/dia']

  energia_bon = periodo_bon * calculo['tc_8h_euros/kwh/hora']
  energia_nbon = periodo_nbon * calculo['tc_16h_euros/kwh/hora']

  subtotal = pot_v + pot_p + energia_bon + energia_nbon

  impuesto = subtotal * (calculo['impuesto_actual']/100)

  contador = formulario['periodo']*calculo['alquiler_equipo_euros/dia']

  total = subtotal + impuesto + contador

  total_factura = total * 1.10

  return total_factura

In [259]:
def tramo_fija(fila):

    datos = fila.to_dict()

    energia_valle = datos['consumo_valle'] * datos['termino_consumo_euros/kwh/hora']
    energia_llano = datos['consumo_llano'] * datos['termino_consumo_euros/kwh/hora']
    energia_punta = datos['consumo_punta'] * datos['termino_consumo_euros/kwh/hora']

    return {'coste_valle': energia_valle,
        'coste_llano': energia_llano,
        'coste_punta': energia_punta}

def tramo_pvpc(fila):

    datos = fila.to_dict() 

    energia_valle = datos['consumo_valle'] * datos['tc_valle_euros/kwh/hora']
    energia_llano = datos['consumo_llano'] * datos['tc_llano_euros/kwh/hora']
    energia_punta = datos['consumo_punta'] * datos['tc_punta_euros/kwh/hora']

    return {'coste_valle': energia_valle,
        'coste_llano': energia_llano,
        'coste_punta': energia_punta}

def tramo_libre(fila):

    datos = fila.to_dict() 

    energia_valle = datos['consumo_valle'] * datos['tc_valle_euros/kwh/hora']
    energia_llano = datos['consumo_llano'] * datos['tc_llano_euros/kwh/hora']
    energia_punta = datos['consumo_punta'] * datos['tc_punta_euros/kwh/hora']

    return {'coste_valle': energia_valle,
        'coste_llano': energia_llano,
        'coste_punta': energia_punta}


def tramo_dh(fila):

    datos = fila.to_dict() 

    energia_valle = datos['consumo_valle'] * datos['tc_8h_euros/kwh/hora']
    energia_llano = datos['consumo_llano'] * datos['tc_16h_euros/kwh/hora']
    energia_punta = datos['consumo_punta'] * datos['tc_16h_euros/kwh/hora']

    return {'coste_valle': energia_valle,
        'coste_llano': energia_llano,
        'coste_punta': energia_punta}

In [260]:
def calcular_total(fila):
    if fila['tipo'] == 'PVPC':
        return pvpc(fila, formulario)
    elif fila['tipo'] == 'TRAMOS':
        return tramos(fila, formulario)
    elif fila['tipo'] == 'DH':
        return dh(fila, formulario)
    else:
        return fija(fila, formulario)

In [261]:
def calcular_tramos(fila):
    if fila['tipo'] == 'PVPC':
        return tramo_pvpc(fila)
    elif fila['tipo'] == 'TRAMOS':
        return tramo_libre(fila)
    elif fila['tipo'] == 'DH':
        return tramo_dh(fila)
    else:
        return tramo_fija(fila)

In [276]:
calculo['total_factura'] = calculo.apply(calcular_total, axis=1)


In [263]:
calculo = calculo.join(calculo.apply(lambda fila: pd.Series(calcular_tramos(fila)), axis=1))

In [264]:
calculo

,id,compania,tarifa,tp_punta_euros/kwh/dia,tp_valle_euros/kwh/dia,peaje_tran_dist_pot_punta,peaje_tran_dist_pot_valle,peaje_ener_valle_tdc,peaje_ener_llano_tdc,peaje_ener_punta_tdc,termino_consumo_euros/kwh/hora,tc_valle_euros/kwh/hora,tc_llano_euros/kwh/hora,tc_punta_euros/kwh/hora,tc_8h_euros/kwh/hora,tc_16h_euros/kwh/hora,mercado,tipo,otros,alquiler_equipo_euros/dia,impuesto_actual,bono_social_euro/dia,margen_euros/kWh/dia,consumo_valle,consumo_llano,consumo_punta,total_actual,total_factura,coste_valle,coste_llano,coste_punta
0,0,Repsol,Mis 10 Horas,0.068192,0.068192,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.000000,0.000000,0.000000,0.086415,0.17283,Libre,DH,0.00,0.026557,5.11,0.0,0.0,100.0,100.0,100.0,10.0,66.444445,8.6415,17.2830,17.2830
1,1,Iberdrola,Plan Elige 8 Horas,0.108192,0.042411,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.000000,0.000000,0.000000,0.096700,0.24100,Libre,DH,0.00,0.026557,5.11,0.0,0.0,100.0,100.0,100.0,10.0,85.024972,9.6700,24.1000,24.1000
2,2,Iberdrola,Plan Noche,0.108192,0.042411,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.000000,0.000000,0.000000,0.109000,0.23900,Libre,DH,0.00,0.026557,5.11,0.0,0.0,100.0,100.0,100.0,10.0,85.984626,10.9000,23.9000,23.9000
3,3,Iberdrola,Plan Familias,0.108192,0.042411,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.000000,0.000000,0.000000,0.103000,0.18900,Libre,DH,0.00,0.026557,5.11,0.0,0.0,100.0,100.0,100.0,10.0,73.728800,10.3000,18.9000,18.9000
4,4,Iberdrola,Plan Verano,0.108192,0.042411,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.000000,0.000000,0.000000,0.099000,0.21900,Libre,DH,0.00,0.026557,5.11,0.0,0.0,100.0,100.0,100.0,10.0,80.203576,9.9000,21.9000,21.9000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,99,Iris Energia,Tarifa estable,0.087374,0.020463,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.149402,0.164994,0.220819,0.000000,0.00000,Libre,TRAMOS,0.00,0.026557,5.11,0.0,0.0,100.0,100.0,100.0,10.0,75.102014,14.9402,16.4994,22.0819
100,100,Eres Energia,Dos punto cero,0.079300,0.019600,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.117000,0.147500,0.204500,0.000000,0.00000,Libre,TRAMOS,0.00,0.026557,5.11,0.0,0.0,100.0,100.0,100.0,10.0,66.423198,11.7000,14.7500,20.4500
101,101,Fortuluz,Precio indexado fijo,0.087955,0.013803,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.105167,0.129574,0.175753,0.000000,0.00000,Libre,TRAMOS,0.00,0.026557,5.11,0.0,0.0,100.0,100.0,100.0,10.0,59.985816,10.5167,12.9574,17.5753
102,102,Fortuluz,A contraluz,0.096962,0.021832,0.008192,0.000527,0.002752,0.027963,0.076974,0.0,0.138047,0.182545,0.231757,0.000000,0.00000,Libre,TRAMOS,0.00,0.026557,5.11,0.0,0.0,100.0,100.0,100.0,10.0,78.337255,13.8047,18.2545,23.1757


In [265]:
minimo = calculo[calculo['total_factura'] == calculo['total_factura'].min()].iloc[0]
minimo

id                                             37
compania                          Integra Energia
tarifa                                      2.0TD
tp_punta_euros/kwh/dia                   0.069566
tp_valle_euros/kwh/dia                    0.02954
peaje_tran_dist_pot_punta                0.008192
peaje_tran_dist_pot_valle                0.000527
peaje_ener_valle_tdc                     0.002752
peaje_ener_llano_tdc                     0.027963
peaje_ener_punta_tdc                     0.076974
termino_consumo_euros/kwh/hora               0.09
tc_valle_euros/kwh/hora                       0.0
tc_llano_euros/kwh/hora                       0.0
tc_punta_euros/kwh/hora                       0.0
tc_8h_euros/kwh/hora                          0.0
tc_16h_euros/kwh/hora                         0.0
mercado                                     Libre
tipo                                         FIJO
otros                                         0.0
alquiler_equipo_euros/dia                0.026557


In [266]:
minimo

id                                             37
compania                          Integra Energia
tarifa                                      2.0TD
tp_punta_euros/kwh/dia                   0.069566
tp_valle_euros/kwh/dia                    0.02954
peaje_tran_dist_pot_punta                0.008192
peaje_tran_dist_pot_valle                0.000527
peaje_ener_valle_tdc                     0.002752
peaje_ener_llano_tdc                     0.027963
peaje_ener_punta_tdc                     0.076974
termino_consumo_euros/kwh/hora               0.09
tc_valle_euros/kwh/hora                       0.0
tc_llano_euros/kwh/hora                       0.0
tc_punta_euros/kwh/hora                       0.0
tc_8h_euros/kwh/hora                          0.0
tc_16h_euros/kwh/hora                         0.0
mercado                                     Libre
tipo                                         FIJO
otros                                         0.0
alquiler_equipo_euros/dia                0.026557


In [267]:
#CODIGO PARA STREAMLIT

print(f'Hemos encontrado esta tarifa: {minimo["tarifa"]} de {minimo["compania"]}')

dif_tarifa = minimo['total_actual'] - minimo['total_factura']

if dif_tarifa < 0:
    print('No existe una mejor tarifa actualmente.')
else:
    print(f'Mejoraría tu tarifa en {dif_tarifa:.2f}€')

Hemos encontrado esta tarifa: 2.0TD de Integra Energia
No existe una mejor tarifa actualmente.


In [268]:
a = 1500 - 9.14
a

1490.86

In [269]:
calculo.to_csv('../Datos/resultados.csv',mode='w')

In [270]:
datos_formulario = pd.read_csv(r'../Datos/datos_tarifas.csv')

datos_formulario['Consumo total (kWh)'] = datos_formulario['Consumo en punta (kWh)'] + datos_formulario['Consumo en llano (kWh)'] + datos_formulario['Consumo en valle (kWh)']


In [271]:
ultimo_consumo_total = datos_formulario['Consumo total (kWh)'].iloc[-1]


In [272]:
ultimo_consumo_total

300.0